# Evaluate semisynthetic SCHOLAR results

In [575]:
# yelp | booking 
data = "yelp"
dataset = "semisynth_" + data
# noCV | cv5050 (CV5050 for Yelp)
train_type = "CV5050"
# ONLY FOR YELP: 1.0 | 0.5 | 0.0
gamma = 0.0

In [576]:
import os, sys, scipy, json
from scipy import sparse
import codecs
import numpy as np
import pandas as pd
import file_handling as fh
from sklearn.model_selection import train_test_split

In [577]:
if sys.platform == "darwin":
    pass
else:
    if data == "booking":
        input_dir = "/nfs/home/maxa/results/"+dataset+'/scholar/' + train_type + "/"    
    elif gamma == 1.0 or gamma == 0.0:
        input_dir = "/nfs/home/maxa/results/"+dataset+'/scholar_gamma'+str(gamma)+ '/' + train_type + "/"    
    elif gamma == 0.5:
        input_dir = "/nfs/home/maxa/results/"+dataset+'/scholar/' + train_type + "/"    
print(input_dir)
#try:
#    os.mkdir(output_dir)
#except:
#pass

/nfs/home/maxa/results/semisynth_yelp/scholar_gamma0.0/CV5050/


In [578]:
print(len(os.listdir(input_dir)))
os.listdir(input_dir)

5


['k20', 'k50', 'k30', 'k10', 'k5']

In [579]:
if data == 'booking':
    ks = ['k2', 'k5', 'k10', 'k15', 'k20', "k25", "k30", "k40", "k50"]
elif data == 'yelp':
    ks = ['k5', 'k10', 'k20', 'k30', 'k50']
    

In [580]:
all_weights = {}
for k in ks:
    n_k = int(k[1:])
    weights = pd.DataFrame(columns = ["w1","w2"])
    runs_dir = input_dir + k
    for run in os.listdir(runs_dir):
        try:
            if train_type == "noCV":
                W_all = pd.read_csv(os.path.join(runs_dir,run,'end_of_training_regression_weights.csv'),
                                   header = 0, index_col = 0).W
            else:
                W_all = pd.read_csv(os.path.join(runs_dir,run,'best_val_regression_weights.csv'),
                                   header = 0, index_col = 0).W
        except:
            continue
        # append weights of across all runs
        W = pd.Series(W_all[n_k:])
        W.index = ["w1","w2"]
        weights = weights.append(W)
    all_weights[k] = weights

In [581]:
cols = ["w1_lower","w1_median","w1_upper","w2_lower","w2_median","w2_upper"]
CI_weights = pd.DataFrame(columns= cols)
for k in all_weights:
    w1_lower = all_weights[k].w1.quantile(0.025)
    w1_median = all_weights[k].w1.quantile(0.5)
    w1_upper = all_weights[k].w1.quantile(0.975)
    
    w2_lower = all_weights[k].w2.quantile(0.025)
    w2_median = all_weights[k].w2.quantile(0.5)
    w2_upper = all_weights[k].w2.quantile(0.975)
    CI_k = pd.DataFrame(data = [w1_lower,w1_median,w1_upper,w2_lower,w2_median,w2_upper]).T
    CI_k.columns = cols
    CI_weights = pd.concat([CI_weights,CI_k], ignore_index=True, axis=0)
temp = CI_weights.T
temp.columns = ks
CI_weights = temp.T
CI_weights

,w1_lower,w1_median,w1_upper,w2_lower,w2_median,w2_upper
k5,-1.031273,-1.006623,-0.959698,0.964653,1.153613,1.237576
k10,-1.019306,-1.007989,-0.990144,1.026818,1.156793,1.189835
k20,-1.025281,-1.004315,-0.980695,0.965354,1.124614,1.171344
k30,-1.006630,-0.994264,-0.979716,1.031266,1.130768,1.155399
k50,-1.026473,-1.004342,-0.972842,0.919885,1.025474,1.153655


In [582]:
all_weights["k50"].shape[0]

20

In [583]:
# = CI_weights.drop("k30")

In [584]:
if data == "booking":
    CI_weights.to_csv("/nfs/home/maxa/results/"+dataset+"/scholar/semisynth_booking_scholar_regweight_bootstrap_"+train_type+".csv")
else:
    CI_weights.to_csv("/nfs/home/maxa/results/"+dataset+"/semisynth_yelp_scholar_regweight_bootstrap_"+train_type+"_gamma"+str(gamma)+".csv")

# Top words

In [171]:
if dataset == 'booking':
    ks = ['k_30', 'k_20', 'k_10', 'k_100', 'k_50']
elif dataset == 'yelp':
    ks = ['k30', 'k20', 'k10', 'k100', 'k50']
    

In [166]:
n_topwords = 10
example_run = 10

In [190]:
k = "k_10"
runs_dir = input_dir + k
run = runs_dir[0]
for k in ks:
    runs_dir = input_dir + k
    run = os.listdir(runs_dir)[example_run]
    os.path.join(runs_dir,run,'/topics.txt')
    pd.read_csv(os.path.join(runs_dir,run,'topics.txt'), header = None)
    

In [194]:
runs_dir + run

'/nfs/home/maxa/results/booking/scholar/k_50k50_ep200_run_12'

In [197]:
pd.read_csv(os.path.join(runs_dir,run,'topics.txt'), header = None)

,0
0,refurbish worn furnitur carpet renov corridor ...
1,noisi nois condit air loud unit neighbor awak ...
2,channel tv news english nespresso comput equip...
3,tea coffe kettl machin maker spotless pod faci...
4,loung tabl bar queue waiter drink relax seat w...
5,bacon egg scrambl bread sausag toast omelett f...
6,slope slipperi socket mood lamp bedsid shelf s...
7,car park public metrono hyde transport garag p...
8,phone data tv mobil speaker awesom devic elect...
9,recommand twice housekeep insist knock excus i...


FileNotFoundError: [Errno 2] No such file or directory: '/topics.txt'

In [166]:
pR = pd.DataFrame()
mse = pd.DataFrame()
pplxy = pd.DataFrame()
for k in ks:
    topwords = []
    runs_dir = input_dir + k
    for run in os.listdir(runs_dir):
        pRs.append(pd.read_csv(os.path.join(runs_dir,run,'topics.txt'),
                               header = None).values[0][0])
        mses.append(pd.read_csv(os.path.join(runs_dir,run,'test_eval_mse.txt'),
                               header = None).values[0][0])
        pplxies.append(pd.read_csv(os.path.join(runs_dir,run,'test_eval_perplexity.txt'),
                               header = None).values[0][0]) 
    pR = pd.concat([pR,pd.DataFrame(pRs, columns=[k])], ignore_index=True, axis=1)
    mse = pd.concat([mse,pd.DataFrame(mses, columns=[k])], ignore_index=True, axis=1)
    pplxy = pd.concat([pplxy,pd.DataFrame(pplxies, columns=[k])], ignore_index=True, axis=1)
pR.columns = ks
mse.columns = ks
pplxy.columns = ks

FileNotFoundError: [Errno 2] No such file or directory: '/nfs/home/maxa/results/booking/scholar/k_200/k200_ep200_run_8/topics.txt'

In [167]:
input_dir + k

'/nfs/home/maxa/results/booking/scholar/k_200'

In [169]:
k = "k_10"

In [170]:
k

'k_10'

# 